In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import census_key
c = Census(census_key, year=2018)

In [2]:
census_data = c.acs5.get(('B01003_001E','B19013_001E','B19301_001E','B02018_007E'), 
                         {'for': 'zip code tabulation area:*'})

census_pd = pd.DataFrame(census_data)

census_pd.head()


,B01003_001E,B19013_001E,B19301_001E,B02018_007E,zip code tabulation area
0,17242.0,13092.0,6999.0,0.0,00601
1,38442.0,16358.0,9277.0,0.0,00602
2,48814.0,16603.0,11307.0,112.0,00603
3,6437.0,12832.0,5943.0,0.0,00606
4,27073.0,19309.0,10220.0,0.0,00610
